In [0]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

#Список компаній 
tickers = ["META", "AAPL", "GOOG", "TSLA", "AMZN", "MSFT", "NVDA"]

#Визначення початкової та кінцевої дати 
start_date = "2020-01-01"
fixed_end_date = "2025-06-10"
today_date = datetime.today().strftime('%Y-%m-%d')
yesterday_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

#Інкрементальне формування файлу
if today_date > fixed_end_date:
    filename = f"stock_prices_{yesterday_date}.csv"
    header_required = False #Відсутній заголовок для щоденних файлів
    data = yf.download(tickers, start=yesterday_date, end=today_date, group_by="ticker")
#Формування історичного файлу з '2020-01-01'до'2025-06-10'
else:
    filename = f"stock_prices_{fixed_end_date}.csv"
    header_required = True #Є заголовок для історичного файлу
    data = yf.download(tickers, start=start_date, end=fixed_end_date, group_by="ticker")
#Додаванання повної назви компанії
company_names = {ticker: yf.Ticker(ticker).info.get('longName', ticker) for ticker in tickers}

formatted_data = {}
for ticker in tickers:
    df = data[ticker].copy()
    df["Company"] = company_names[ticker]  
    formatted_data[ticker] = df
#Об'єднання даних з назвою
final_df = pd.concat(formatted_data.values())

final_df.reset_index(inplace=True)  
final_df.rename(columns={"index": "Date"}, inplace=True) 
#Вибір даних для збереження
final_df = final_df[["Date","Company","Open","High" ,"Low", "Close", "Volume"]]
#Збереження csv файлу, header залежить від даних історичні або щоденні 
final_df.to_csv(filename, index=False, header=header_required)
